In [10]:
#install pyown
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [34]:
# Scrping data from hofstedes website
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = 'https://www.hofstede-insights.com/country-comparison-tool'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
# find the class: "l-comparison__content__item-multiple__top"
table = soup.find('div', attrs={'class': 'l-comparison__content__item-multiple__top'})
# loop through all classes c-overview fadeInUp
culture_dict = {}
for row in table.findAll('div', attrs={'class': 'c-overview fadeInUp'}):
    # save country name as key and score as value
    country_scores = []
    # save country name within data-country in dictionary as key
    country = row['data-country']
    # power-distance score in span class
    power_distance = row.find('span', attrs={'class': 'power-distance fadeIn active'}).text
    individualism = row.find('span', attrs={'class': 'individualism fadeIn'}).text
    motivation = row.find('span', attrs={'class': 'motivation fadeIn'}).text
    uncertainty_avoidance = row.find('span', attrs={'class': 'uncertainty-avoidance fadeIn'}).text
    long_term_orientation = row.find('span', attrs={'class': 'long-term-orientation fadeIn'}).text
    indulgence = row.find('span', attrs={'class': 'indulgence fadeIn'}).text

    # save country name as key and score as value
    country_scores.append(power_distance)
    country_scores.append(individualism)
    country_scores.append(motivation)
    country_scores.append(uncertainty_avoidance)
    country_scores.append(long_term_orientation)
    country_scores.append(indulgence)
    if country not in culture_dict:
        culture_dict[country] = country_scores
    
# create dataframe from dictionary
culture_df = pd.DataFrame.from_dict(culture_dict, orient='index', columns=['power_distance', 'individualism', 'motivation', 'uncertainty_avoidance', 'long_term_orientation', 'indulgence'])
culture_df.head()

# save dataframe to csv in the data cleaned folder
culture_df.to_csv('../data/cleaned/culture_df.csv')





In [12]:

# Load the data from the CSV file
file_path = '../data/cleaned/510df.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()


# Dropping any columns with non-numeric data for simplicity.
numeric_data = data.select_dtypes(include=['number']).copy()

# Filling missing values with the mean of their respective columns.
numeric_data.fillna(numeric_data.mean(), inplace=True)

# Defining the features and target variable
X = numeric_data.drop(columns=['Life expectancy '])  # Assuming 'Life expectancy ' is the target column
y = numeric_data['Life expectancy ']


# Feature selection using Random Forest
selector = SelectFromModel(estimator=RandomForestRegressor(n_estimators=100, random_state=42)).fit(X_train, y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Training the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_selected, y_train)

# Predicting and evaluating the model
predictions = model.predict(X_test_selected)
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5

# Outputting the selected feature names and the model performance
selected_features = X.columns[(selector.get_support())]
rmse, selected_features.tolist()


# Removing features that give direct information about life expectancy
irrelevant_features = [
    'Life expectancy at birth, female (years)',
    'Life expectancy at birth, male (years)',
    'Life expectancy at birth, total (years)',
    'Survival to age 65, female (% of cohort)',
    'Adult Mortality'
]


# Updating the list of selected features by removing the irrelevant ones
selected_features = [feature for feature in selected_features if feature not in irrelevant_features]

# Scaling the features
scaler = StandardScaler()

# Retraining the model with the final set of features
X_final = X[selected_features]
X_final_scaled = scaler.fit_transform(X_final)

# Splitting the final dataset into training and testing sets
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_final_scaled, y, test_size=0.2, random_state=42)

# Final model training
final_model = RandomForestRegressor(n_estimators=100, random_state=42)
final_model.fit(X_train_final, y_train_final)

# Final predictions
final_predictions = final_model.predict(X_test_final)

# Validation metrics
final_mse = mean_squared_error(y_test_final, final_predictions)
final_rmse = np.sqrt(final_mse)
final_mae = mean_absolute_error(y_test_final, final_predictions)
final_r2 = r2_score(y_test_final, final_predictions)

# Well-formatted output
output = {
    'Root Mean Squared Error (RMSE)': final_rmse,
    'Mean Absolute Error (MAE)': final_mae,
    'R-squared (R2 Score)': final_r2,
    'Features Used for Prediction': selected_features
}


ValueError: could not convert string to float: 'MALDIVES'